# Open double-funded channel without intermediary

In [ ]:
%load_ext autoreload
%autoreload 2

## Setup LND clients for Ali and Bob

In [ ]:
import sys

In [ ]:
# LND GRPC bindings should be in '/home/jovyan/lnrpc'
sys.path.append('/home/jovyan/lnrpc')

In [ ]:
%%sh
sudo cp /ali-lnd/data/chain/bitcoin/regtest/admin.macaroon /tmp/ali.macaroon
sudo chmod +r /tmp/ali.macaroon

sudo cp /bob-lnd/data/chain/bitcoin/regtest/admin.macaroon /tmp/bob.macaroon
sudo chmod +r /tmp/bob.macaroon

In [ ]:
from p2oc.lnd_rpc import LndRpc, lnmsg, walletmsg, signrpc, signmsg, routermsg

In [ ]:
ali = LndRpc(host='ali-lnd:10009',
             cert_path='/ali-lnd/tls.cert',
             macaroon_path='/tmp/ali.macaroon')

In [ ]:
bob = LndRpc(host='bob-lnd:10009',
             cert_path='/bob-lnd/tls.cert',
             macaroon_path='/tmp/bob.macaroon')

In [ ]:
aliNodePubKey = ali.lnd.GetInfo(lnmsg.GetInfoRequest()).identity_pubkey
aliNodePubKey

In [ ]:
bobNodePubKey = bob.lnd.GetInfo(lnmsg.GetInfoRequest()).identity_pubkey
bobNodePubKey

## Fund Ali and Bob

In [ ]:
import os
import bitcoin
from p2oc.btc_rpc import Proxy, Config

In [ ]:
bitcoin.SelectParams('regtest')

In [ ]:
# only used to mine initial coins for Ali and Bob
brpc = Proxy(config=Config(
    rpcuser=os.environ['BTCD_RPCUSER'],
    rpcpassword=os.environ['BTCD_RPCPASS'],
    rpcconnect='bitcoind',
    rpcport=18443
))

In [ ]:
brpc.getblockcount()

In [ ]:
aliAddr = ali.lnd.NewAddress(lnmsg.NewAddressRequest(type=0)) # p2wkh
aliAddr

In [ ]:
bobAddr = bob.lnd.NewAddress(lnmsg.NewAddressRequest(type=0)) # p2wkh
bobAddr

In [ ]:
# fund ali
_ = list(brpc.generatetoaddress(1, aliAddr.address))

In [ ]:
# fund bob
_ = list(brpc.generatetoaddress(1, bobAddr.address))

In [ ]:
brpc.createwallet('miner')
minerAddr = brpc.getnewaddress("coinbase")
brpc.unloadwallet('miner')

In [ ]:
# unlock mined coins
_ = list(brpc.generatetoaddress(110, minerAddr))

In [ ]:
ali.lnd.WalletBalance(lnmsg.WalletBalanceRequest())

In [ ]:
bob.lnd.WalletBalance(lnmsg.WalletBalanceRequest())

### Create dummy address

In [ ]:
import hashlib
import time
import base64

import bitcoin.core.script as bs
import bitcoin.wallet as bw

In [ ]:
# create dummy P2WPKH address to call FundPsbt api
h = hashlib.sha256(b'correct horse battery staple').digest()
seckey = bw.CBitcoinSecret.from_secret_bytes(h)

# Create an address from that private key.
public_key = seckey.pub
scriptPubKey = bs.CScript([bs.OP_0, bs.Hash160(public_key)])
dummy_address = bw.P2WPKHBitcoinAddress.from_scriptPubKey(scriptPubKey)
dummy_address

## Ali creates public offer to request inbound liquidity in exchange for a fee

In [ ]:
from bitcointx.core.psbt import PartiallySignedTransaction, PSBT_ProprietaryTypeData, PSBT_Output
import bitcointx.core as bc
import bitcointx.core.script as bs
COIN = bc.CoreCoinParams.COIN

from bitcoin.rpc import hexlify, unhexlify

# NOTE: bitcoin and bitcointx don't play well together

### Allocate funds

In [ ]:
# taker needs to pay premium to open channel
premiumAmount = int(0.001 * COIN) # premium Ali is willing to pay
fundAmount = int(0.16 * COIN) # requested inbound capacity

In [ ]:
# create dummy psbt to extract 'funding' UTXO and change addresses
tx_template = walletmsg.TxTemplate(outputs={str(dummy_address): premiumAmount})

psbtRequest = walletmsg.FundPsbtRequest(raw=tx_template, target_conf=6)

In [ ]:
psbt = ali.wallet.FundPsbt(request=psbtRequest)
psbt

In [ ]:
change_output_index = psbt.change_output_index

In [ ]:
# convert to python psbt object
psbt = PartiallySignedTransaction.from_binary(psbt.funded_psbt)

In [ ]:
# remove dummy output
assert len(psbt.unsigned_tx.vout) == 2, 'expect "dummy" and "change" outputs'

# copy tx object
tx = bc.CMutableTransaction.from_instance(psbt.unsigned_tx)
# add change output
tx.vout = [psbt.unsigned_tx.vout[change_output_index]]

original_fee = psbt.get_fee()

# update psbt
psbt.unsigned_tx = tx
psbt.outputs = [psbt.outputs[change_output_index]]
psbt.outputs[0].index = 0

# check that "dummy" output for premium was removed 
assert psbt.get_fee() == original_fee + premiumAmount

### Create offer

In [ ]:
import json

In [ ]:
# Generate pubkey for channel funding
key_family = 0 # multisig?
keyReq = walletmsg.KeyReq(key_family=key_family)
aliKeyDesc = ali.wallet.DeriveNextKey(keyReq)
# aliKeyDesc = ali.wallet.DeriveNextKey(keyReq) # need to call twice?
aliKeyDesc

In [ ]:
offerParams = {
    'type': 'INBOUND_LIQUIDITY_REQUEST',
    'nodeAddr': 'ali-lnd',
    'nodePubKey': aliNodePubKey,
    'premiumAmount': premiumAmount,
    'fundAmount': fundAmount,
    'chanPubKey1': hexlify(aliKeyDesc.raw_key_bytes)
}
offerParams

In [ ]:
psbt.proprietary_fields[b'offer'] = [
    PSBT_ProprietaryTypeData(0, b'params', json.dumps(offerParams).encode()),
    PSBT_ProprietaryTypeData(0, b'signature', b'TODO ADD SIGNATURE'),
]

In [ ]:
offer = psbt.to_base64()
del psbt # delete psbt so we don't accidentally leak some state to the other party
offer

## Bob accepts offer and sends reply

### Check that the offer is still valid

In [ ]:
psbt1 = PartiallySignedTransaction.from_base64(offer)
psbt1

In [ ]:
offerParams = psbt1.proprietary_fields[b'offer'][0].value
offerParams = json.loads(offerParams)
offerParams

In [ ]:
# check that funding UTXOs has not been spent
# note we can't use `lnd.GetTransactions` since it only knows about our wallet's transactions
# it's probably safe to skip this step because blockchain will prevent from double spending
for vin in psbt1.unsigned_tx.vin:
    utxo = brpc.gettxout(vin.prevout)
    assert utxo is not None

In [ ]:
# for simplicity Bob is maker but it can take "taker" role as well
assert offerParams['type'] == 'INBOUND_LIQUIDITY_REQUEST'

In [ ]:
premiumAmount = offerParams['premiumAmount'] # Ali's premium to Bob
fundAmount = offerParams['fundAmount'] # Bob funds this amount

In [ ]:
feesAmount = psbt1.get_fee() - premiumAmount
assert feesAmount > 0
feesAmount

### Allocate funds

In [ ]:
# create dummy psbt to extract 'funding' UTXO and change addresses
# user minerAddr as a dummy address to create psbt
tx_template = walletmsg.TxTemplate(outputs={str(dummy_address): fundAmount})

psbtRequest = walletmsg.FundPsbtRequest(raw=tx_template, target_conf=6)

In [ ]:
psbt2 = bob.wallet.FundPsbt(request=psbtRequest)
psbt2

In [ ]:
change_output_index = psbt2.change_output_index

In [ ]:
# convert to python psbt object
psbt2 = PartiallySignedTransaction.from_binary(psbt2.funded_psbt)

In [ ]:
# remove dummy output
assert len(psbt2.unsigned_tx.vout) == 2, 'expect "dummy" and "change" outputs'

# copy tx object
tx = bc.CMutableTransaction.from_instance(psbt2.unsigned_tx)
# add change output
tx.vout = [psbt2.unsigned_tx.vout[change_output_index]]

original_fee = psbt2.get_fee()

# update psbt
psbt2.unsigned_tx = tx
psbt2.outputs = [psbt2.outputs[change_output_index]]
psbt2.outputs[0].index = 0

# check that "dummy" output for fundAmount was removed 
assert psbt2.get_fee() == original_fee + fundAmount

### Add our inputs and outputs to the offer psbt

In [ ]:
for i, vin in enumerate(psbt2.unsigned_tx.vin):
    inp = psbt2.inputs[i]
    inp.index = None # reset index
    psbt1.add_input(vin, inp)

In [ ]:
for i, vout in enumerate(psbt2.unsigned_tx.vout):
    out = psbt2.outputs[i]
    out.index = None # reset index
    psbt1.add_output(vout, out)

### Create funding output

In [ ]:
key_family = 0 # multisig?
keyReq = walletmsg.KeyReq(key_family=key_family)
bobKeyDesc = bob.wallet.DeriveNextKey(keyReq)
# bobKeyDesc = bob.wallet.DeriveNextKey(keyReq) # need to call twice?
bobKeyDesc

In [ ]:
takerPubKey = unhexlify(offerParams['chanPubKey1'])
makerPubKey = bobKeyDesc.raw_key_bytes

In [ ]:
# https://github.com/lightningnetwork/lightning-rfc/blob/master/03-transactions.md#funding-transaction-output
# https://github.com/bitcoin/bips/blob/master/bip-0069.mediawiki
# if makerPubKey > takerPubKey:
if list(makerPubKey) > list(takerPubKey):
    pk1, pk2 = takerPubKey, makerPubKey
else:
    pk1, pk2 = makerPubKey, takerPubKey

In [ ]:
msigScript = bs.CScript([
        bs.OP_2,
        pk1,
        pk2,
        bs.OP_2,
        bs.OP_CHECKMULTISIG
    ])

In [ ]:
# convert to P2WSH
scriptPubKey = bs.CScript([bs.OP_0, hashlib.sha256(msigScript).digest()])
scriptPubKey

In [ ]:
assert scriptPubKey.is_witness_v0_scripthash()

In [ ]:
fundingOutput = bc.CTxOut(premiumAmount + fundAmount,
                          scriptPubKey)
fundingOutput

In [ ]:
# add funding output to the original psbt
# must be the last one
psbt1.add_output(fundingOutput, PSBT_Output())

In [ ]:
assert psbt1.get_output_amounts()[-1] == fundAmount + premiumAmount

In [ ]:
psbt1.get_input_amounts()

In [ ]:
# TODO: currently both parties pay fees. It probably makes sense that only Ali pays fees
psbt1.get_fee()

### connect to taker (Ali)

In [ ]:
# assumer whoever creates offer is accessible
connect_peer_req = lnmsg.ConnectPeerRequest(
    addr=lnmsg.LightningAddress(
        pubkey=offerParams['nodePubKey'],
        host=offerParams['nodeAddr']
))

In [ ]:
bob.lnd.ConnectPeer(connect_peer_req)

In [ ]:
# check that we are connected
bob.lnd.ListPeers(lnmsg.ListPeersRequest())

### Send Reply

In [ ]:
replyParams = {
    
}

In [ ]:
psbt.proprietary_fields[b'reply'] = [
    PSBT_ProprietaryTypeData(0, b'params', json.dumps(offerParams).encode()),
    PSBT_ProprietaryTypeData(0, b'signature', b'TODO ADD SIGNATURE'),
]

## Open pending channel

In [ ]:
fundingTxId = psbt1.unsigned_tx.GetTxid()
fundingOutputIdx = len(psbt1.unsigned_tx.vout) - 1 # last output

In [ ]:
bc.b2lx(fundingTxId)

In [ ]:
# something that is unique to the corresponding matched taker and maker
channelId = hashlib.sha256((bobNodePubKey + offerParams['nodePubKey'] + str(time.time())).encode()).hexdigest()
channelId

In [ ]:
chan_point = lnmsg.ChannelPoint(funding_txid_bytes=fundingTxId,
                                output_index=fundingOutputIdx)

In [ ]:
# our funding output key
local_key = lnmsg.KeyDescriptor(
    raw_key_bytes=bobKeyDesc.raw_key_bytes,
    key_loc=lnmsg.KeyLocator(
        key_family=bobKeyDesc.key_loc.key_family,
        key_index=bobKeyDesc.key_loc.key_index
    )
)

In [ ]:
chan_point_shim = lnmsg.ChanPointShim(
    amt=fundAmount + premiumAmount,
    chan_point=chan_point,
    local_key=local_key,
    remote_key=unhexlify(offerParams['chanPubKey1']),
    pending_chan_id=unhexlify(channelId),
    thaw_height=0 # for simplicity
)

In [ ]:
open_chan_req = lnmsg.OpenChannelRequest(
    node_pubkey=unhexlify(offerParams['nodePubKey']),
    local_funding_amount=fundAmount + premiumAmount,
    push_sat=premiumAmount, # premiumAmount is pushed to the remote end (Ali)
    target_conf=6,
    funding_shim=lnmsg.FundingShim(chan_point_shim=chan_point_shim)
)

In [ ]:
chanEventStream = bob.lnd.OpenChannel(open_chan_req)

In [ ]:
next(chanEventStream)

In [ ]:
# check that the channel is pending
bob.lnd.PendingChannels(lnmsg.PendingChannelsRequest())

In [ ]:
ali.lnd.PendingChannels(lnmsg.PendingChannelsRequest())

### Register shim to prepare for channel opening

In [ ]:
# something that is unique to the corresponding matched taker and maker
channelId = hashlib.sha256((bobNodePubKey + offer['nodePubKey'] + str(time.time())).encode()).hexdigest()
channelId

In [ ]:
chan_point = lnmsg.ChannelPoint(funding_txid_bytes=fundingTxId,
                                output_index=fundingOutputIdx)

In [ ]:
# define our key for funding output
local_key = lnmsg.KeyDescriptor(
    raw_key_bytes=bobKeyDesc.raw_key_bytes,
    key_loc=lnmsg.KeyLocator(
        key_family=bobKeyDesc.key_loc.key_family,
        key_index=bobKeyDesc.key_loc.key_index))

In [ ]:
chan_point_shim = lnmsg.ChanPointShim(
    amt=fundAmount + premiumAmount,
    chan_point=chan_point,
    local_key=local_key,
    remote_key=offer['chanPubKey1'],
    pending_chan_id=unhexlify(channelId),
    thaw_height=0 # set 0 for simplicity
)

In [ ]:
ftm = lnmsg.FundingTransitionMsg(
    shim_register=lnmsg.FundingShim(chan_point_shim=chan_point_shim))

In [ ]:
# empty response
bob.lnd.FundingStateStep(ftm)

### Sign transaction

In [ ]:
# At this point things look good and we can sign the transaction

In [ ]:
# assume single input for simplicity
assert len(bobDummyPsbtDecoded['inputs']) == 1

In [ ]:
input_ = bobDummyPsbtDecoded['inputs'][0]
input_

In [ ]:
# our UTXO pubkey
assert len(input_['bip32_derivs']) == 1
pubkey = input_['bip32_derivs'][0]['pubkey']
pubkey = unhexlify(pubkey)
pubkey

In [ ]:
scriptPubKey = input_['witness_utxo']['scriptPubKey']['hex']
scriptPubKey = bs.CScript(unhexlify(scriptPubKey))

utxoAddr = bw.CBitcoinAddress.from_scriptPubKey(scriptPubKey)
utxoAddr

In [ ]:
sign_desc = signmsg.SignDescriptor(
    key_desc=signmsg.KeyDescriptor(
        raw_key_bytes=pubkey
    ),
    witness_script=utxoAddr.to_redeemScript(),
    output=signmsg.TxOut(
        value=int(input_['witness_utxo']['amount'] * bc.COIN),
        pk_script=utxoAddr.to_scriptPubKey()
    ),
    sighash=bs.SIGHASH_ALL, # so that transaction cannot be altered
    input_index=0 # for simplicity Bob's input is 0
)

In [ ]:
signReq = signmsg.SignReq(
    raw_tx_bytes=fundingTx.serialize(),
    sign_descs=[sign_desc]
)

In [ ]:
bobSignResp = bob.signer.SignOutputRaw(signReq)
bobSignResp

In [ ]:
signature = bobSignResp.raw_sigs[0] + bytes([bs.SIGHASH_ALL])

In [ ]:
witness = [signature, pubkey]
bs.CScriptWitness(witness)

In [ ]:
bobSignedFundingTx = bc.CTransaction(
    vin=fundingTx.vin,
    vout=fundingTx.vout,
    witness=bc.CTxWitness([bc.CTxInWitness(bs.CScriptWitness(witness))]))

### Send offer reply

In [ ]:
offerReply = {
    'id': offer['id'],
    'nodeAddr': 'bob-lnd', # sending for extra check
    'nodePubKey': bobNodePubKey,
    'channelId': channelId,
    'chanPubKey2': bobKeyDesc.raw_key_bytes,
    'fundingOutputIdx': fundingOutputIdx,
    'fundingTx': bobSignedFundingTx
}
offerReply

## Ali checks offer reply and commits funding tx

### Check offer reply message

In [ ]:
# check id
assert offer['id'] == offerReply['id']

In [ ]:
# check connection
peers = ali.lnd.ListPeers(lnmsg.ListPeersRequest())
assert peers.peers[0].pub_key == offerReply['nodePubKey']
# we can also check `nodeAddr`

### Check transaction

In [ ]:
tx = offerReply['fundingTx']

# check that our input was included. For simplicity assume we are the last
assert tx.vin[-1] == offer['inputs'][0]

# check that our change output was included
assert tx.vout[-1] == offer['change']

# check that signature is valid. For simplicity just check that it's present
assert tx.wit is not None

### Check funding output

In [ ]:
premiumAmount = offer['premiumAmount']
fundAmount = offer['fundAmount']

In [ ]:
takerPubKey = offer['chanPubKey1'] # ali
makerPubKey = offerReply['chanPubKey2'] # bob

In [ ]:
# https://github.com/lightningnetwork/lightning-rfc/blob/master/03-transactions.md#funding-transaction-output
# https://github.com/bitcoin/bips/blob/master/bip-0069.mediawiki
if list(makerPubKey) > list(takerPubKey):
    pk1, pk2 = takerPubKey, makerPubKey
else:
    pk1, pk2 = makerPubKey, takerPubKey

In [ ]:
msigScript = bs.CScript([
        bs.OP_2,
        pk1,
        pk2,
        bs.OP_2,
        bs.OP_CHECKMULTISIG
    ])

In [ ]:
# convert to P2WSH
scriptPubKey = bc.CScript([bs.OP_0, hashlib.sha256(msigScript).digest()])
assert scriptPubKey.is_witness_v0_scripthash()

In [ ]:
fundingOutput = bc.CTxOut(premiumAmount + fundAmount,
                          scriptPubKey)
assert fundingOutput == tx.vout[0]

### Open pending channel

In [ ]:
offerReply['fundingTx']

In [ ]:
chan_point = lnmsg.ChannelPoint(funding_txid_bytes=offerReply['fundingTx'].GetTxid(),
                                output_index=offerReply['fundingOutputIdx'])

In [ ]:
# our funding key
local_key = lnmsg.KeyDescriptor(
    raw_key_bytes=aliKeyDesc.raw_key_bytes,
    key_loc=lnmsg.KeyLocator(
        key_family=aliKeyDesc.key_loc.key_family,
        key_index=aliKeyDesc.key_loc.key_index
    )
)

In [ ]:
premiumAmount = offer['premiumAmount']
fundAmount = offer['fundAmount']

In [ ]:
chan_point_shim = lnmsg.ChanPointShim(
    amt=offer['fundAmount'] + offer['premiumAmount'],
    chan_point=chan_point,
    local_key=local_key,
    remote_key=offerReply['chanPubKey2'],
    pending_chan_id=unhexlify(offerReply['channelId']),
    thaw_height=0 # for simplicity
)

In [ ]:
funding_shim = lnmsg.FundingShim(chan_point_shim=chan_point_shim)

In [ ]:
open_chan_req = lnmsg.OpenChannelRequest(
    node_pubkey=unhexlify(offerReply['nodePubKey']),
    local_funding_amount=offer['fundAmount'] + offer['premiumAmount'],
    push_sat=offer['fundAmount'], # fund amount is pushed to the remote end (Bob)
    funding_shim=funding_shim
)

In [ ]:
chanEventStream = ali.lnd.OpenChannel(open_chan_req)

In [ ]:
next(chanEventStream)

In [ ]:
# check that the channel is pending
ali.lnd.PendingChannels(lnmsg.PendingChannelsRequest())

### Sign funding tx

In [ ]:
# for simplicity assume single input
assert len(aliDummyPsbtDecoded['inputs']) == 1

In [ ]:
# for simplicity assume single signing key
assert len(aliDummyPsbtDecoded['inputs'][0]['bip32_derivs']) == 1

In [ ]:
input_ = aliDummyPsbtDecoded['inputs'][0]
input_

In [ ]:
scriptPubKey = input_['witness_utxo']['scriptPubKey']['hex']
scriptPubKey = bs.CScript(unhexlify(scriptPubKey))

utxoAddr = bw.CBitcoinAddress.from_scriptPubKey(scriptPubKey)
utxoAddr

In [ ]:
# our UTXO pubkey
assert len(input_['bip32_derivs']) == 1
pubkey = input_['bip32_derivs'][0]['pubkey']
pubkey = unhexlify(pubkey)
pubkey

In [ ]:
# create copy of other party's signed transaction
finalFundingTx = bc.CMutableTransaction(
    vin=offerReply['fundingTx'].vin,
    vout=offerReply['fundingTx'].vout
)

In [ ]:
sign_desc = signmsg.SignDescriptor(
    key_desc=signmsg.KeyDescriptor(
        raw_key_bytes=pubkey
    ),
    witness_script=utxoAddr.to_redeemScript(),
    output=signmsg.TxOut(
        value=int(input_['witness_utxo']['amount'] * bc.COIN),
        pk_script=utxoAddr.to_scriptPubKey()
    ),
    sighash=bs.SIGHASH_ALL, # so that transaction cannot be altered
    input_index=len(fundingTx.vin) - 1 # assume we are the last one
)

In [ ]:
signReq = signmsg.SignReq(
    raw_tx_bytes=finalFundingTx.serialize(),
    sign_descs=[sign_desc]
)

In [ ]:
aliSignResp = ali.signer.SignOutputRaw(signReq)
aliSignResp

In [ ]:
signature = aliSignResp.raw_sigs[0] + bytes([bs.SIGHASH_ALL])

In [ ]:
witness = [signature, pubkey]
witness = bc.CTxInWitness(bs.CScriptWitness(witness))
witness

In [ ]:
# combine signatures
finalFundingTx.wit = bc.CTxWitness([offerReply['fundingTx'].wit.vtxinwit[0], witness])

### Submit funding transaction

In [ ]:
fundingTxId2 = brpc.sendrawtransaction(finalFundingTx)
fundingTxId2

In [ ]:
assert fundingTxId2 == fundingTxId

In [ ]:
_ = list(brpc.generatetoaddress(6, minerAddr))

### Check that the channel has been opened

In [ ]:
# should be no pending channels
ali.lnd.PendingChannels(lnmsg.PendingChannelsRequest())

In [ ]:
ali.lnd.ListChannels(lnmsg.ListChannelsRequest())